In [1]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers

import time 

Using TensorFlow backend.


### Data Processing

In [2]:
df = pd.read_csv('../data/num_data.csv')

In [3]:
POLLUTION = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']

In [4]:
WEATHER = ['PM2.5', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM']

In [5]:
dataset = df

In [6]:
dataset.shape

(1000, 16)

In [7]:
# Useful functions
def plot_predictions(test, predicted):
    plt.figure(figsize=(30, 15));

    plt.plot(test, color='red', alpha=0.5, label='Actual PM2.5 Concentration',)
    plt.plot(predicted, color='blue', alpha=0.5, label='Predicted PM2.5 Concentation')
    plt.title('PM2.5 Concentration Prediction')
    plt.xlabel('Time')
    plt.ylabel('PM2.5  Concentration')
    plt.legend()
    plt.show()
    

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    return rmse

In [8]:
data_size = dataset.shape[0]
train_size=int(data_size * 0.6)
test_size = 100
valid_size = data_size - train_size - test_size

test_next_day = [12, 24, 48]
n_feature = dataset.shape[1]

In [9]:
training_set = dataset[:train_size].values
valid_set = dataset[train_size:train_size+valid_size].values
test_set = dataset[data_size-test_size:].values

In [10]:
y = dataset.iloc[:,0].values
y = y.reshape(-1,1)

y.shape

(1000, 1)

In [11]:
# Scaling the dataset
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
valid_set_scaled = sc.fit_transform(valid_set)
test_set_scaled = sc.fit_transform(test_set)

sc_y = MinMaxScaler(feature_range=(0,1))
y_scaled = sc_y.fit_transform(y)

In [12]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X_, y_ = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix-1:out_end_ix, 0]
        X_.append(seq_x)
        y_.append(seq_y)
    return np.array(X_), np.array(y_)

In [13]:
n_steps_in = 24
n_steps_out = 24
X_train, y_train = split_sequences(training_set_scaled, n_steps_in, n_steps_out)
X_valid, y_valid = split_sequences(valid_set_scaled, n_steps_in, n_steps_out)
X_test, y_test = split_sequences(test_set_scaled, n_steps_in, n_steps_out)

## Grid Search Control 


In [14]:
n_activation = ['tanh', 'sigmoid', 'relu']
act = n_activation[0]

n_learn_rate = [0.01, 0.1, 0.5]
lr = n_learn_rate[0]

n_optimizers = [optimizers.Adam(lr=lr), optimizers.RMSprop(lr=lr), optimizers.SGD(lr=lr)]
opt = n_optimizers[0]

n_epoches = [50, 100, 200]
epoch = n_epoches[0]

n_batch_size = [32, 256, 1024]
batch = n_batch_size[0]

n_of_neurons = [50, 100, 200]
neuron = n_of_neurons[0]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
X_test_24 = X_test[:24]
y_test_24 = y_test[:24]
rmse_df = pd.DataFrame(columns=['Model', 'train_rmse', 'valid_rmse', '24h_pred_rmse', 'train_time', 'activation', 
                               'learn_rate', 'optimizers'])

In [17]:
for act in n_activation:
    for lr in n_learn_rate:
        n_optimizers = [optimizers.Adam(lr=lr), optimizers.RMSprop(lr=lr), optimizers.SGD(lr=lr)]
        for opt in n_optimizers:
                        
            LSTM_GRU_reg = Sequential()
            LSTM_GRU_reg.add(LSTM(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act))
            LSTM_GRU_reg.add(GRU(units=neuron, activation=act))
            LSTM_GRU_reg.add(Dense(units=n_steps_out))
            LSTM_GRU_reg.compile(optimizer=opt,loss='mean_squared_error')
            
            regressor = LSTM_GRU_reg
            model = 'LSTM_GRU_reg'
    
            print('training start for', model)    
            start = time.process_time()
            regressor.fit(X_train,y_train,epochs=epoch,batch_size=batch)
            train_time = round(time.process_time() - start, 2)

            print('results for training set')
            y_train_pred = regressor.predict(X_train)
            train_rmse = return_rmse(y_train,y_train_pred)

            print('results for valid set')
            y_valid_pred = regressor.predict(X_valid)
            valid_rmse = return_rmse(y_valid,y_valid_pred)

            print('results for test set - 24 hours')
            y_test_pred24 = regressor.predict(X_test_24)
            test24_rmse = return_rmse(y_test_24,y_test_pred24)
    
    
            one_df = pd.DataFrame([[model, train_rmse, valid_rmse, test24_rmse, train_time, opt,lr, act]],
                          columns=['Model', 'train_rmse', 'valid_rmse', '24h_pred_rmse', 'train_time', 'activation', 
                               'learn_rate', 'optimizers'])
            rmse_df = pd.concat([rmse_df, one_df])

# save the rmse results 
rmse_df.to_csv('../rmse_result_grid_v2.csv')

            

training start for LSTM_GRU_reg
Epoch 1/5
554/554 [==============================] - 2s 3ms/step - loss: 0.0036
Epoch 2/5
554/554 [==============================] - 0s 483us/step - loss: 1.2913e-04
Epoch 3/5
554/554 [==============================] - 0s 471us/step - loss: 3.5795e-05
Epoch 4/5
554/554 [==============================] - 0s 479us/step - loss: 1.8527e-05
Epoch 5/5
554/554 [==============================] - 0s 515us/step - loss: 1.2932e-05
results for training set
results for valid set
results for test set - 24 hours


/Users/harry.li/Documents/Condition_Insight/mat.ci.febat.popeye/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



training start for LSTM_GRU_reg
Epoch 1/5
554/554 [==============================] - 2s 3ms/step - loss: 0.0198
Epoch 2/5
554/554 [==============================] - 0s 456us/step - loss: 6.7655e-04
Epoch 3/5
554/554 [==============================] - 0s 447us/step - loss: 7.5593e-04
Epoch 4/5
554/554 [==============================] - 0s 451us/step - loss: 5.1306e-04
Epoch 5/5
554/554 [==============================] - 0s 469us/step - loss: 5.6629e-04
results for training set
results for valid set
results for test set - 24 hours


/Users/harry.li/Documents/Condition_Insight/mat.ci.febat.popeye/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



training start for LSTM_GRU_reg
Epoch 1/5
554/554 [==============================] - 2s 3ms/step - loss: 0.0125
Epoch 2/5
554/554 [==============================] - 0s 527us/step - loss: 0.0097
Epoch 3/5
554/554 [==============================] - 0s 476us/step - loss: 0.0080
Epoch 4/5
554/554 [==============================] - 0s 514us/step - loss: 0.0069
Epoch 5/5
554/554 [==============================] - 0s 464us/step - loss: 0.0061
results for training set
results for valid set
results for test set - 24 hours


...

/Users/harry.li/Documents/Condition_Insight/mat.ci.febat.popeye/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



training start for LSTM_GRU_reg
Epoch 1/5
554/554 [==============================] - 3s 5ms/step - loss: 30427684369.2267
Epoch 2/5
554/554 [==============================] - 0s 500us/step - loss: 0.2099
Epoch 3/5
554/554 [==============================] - 0s 497us/step - loss: 0.2453
Epoch 4/5
554/554 [==============================] - 0s 494us/step - loss: 0.2523
Epoch 5/5
554/554 [==============================] - 0s 500us/step - loss: 0.2534
results for training set
results for valid set
results for test set - 24 hours


/Users/harry.li/Documents/Condition_Insight/mat.ci.febat.popeye/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



training start for LSTM_GRU_reg
Epoch 1/5
554/554 [==============================] - 3s 5ms/step - loss: nan
Epoch 2/5
554/554 [==============================] - 0s 495us/step - loss: nan
Epoch 3/5
554/554 [==============================] - 0s 499us/step - loss: nan
Epoch 4/5
554/554 [==============================] - 0s 494us/step - loss: nan
Epoch 5/5
554/554 [==============================] - 0s 497us/step - loss: nan
results for training set


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
GRU_reg = Sequential()
LSTM_reg = Sequential()
GRU_GRU_reg =  Sequential()
GRU_LSTM_reg = Sequential()
LSTM_GRU_reg = Sequential()
LSTM_LSTM_reg = Sequential()


GRU_reg.add(GRU(units=neuron, input_shape=(X_train.shape[1],n_feature), activation=act))
# The output layer
GRU_reg.add(Dense(units=n_steps_out))


LSTM_reg.add(LSTM(units=neuron, input_shape=(X_train.shape[1],n_feature), activation=act))
LSTM_reg.add(Dense(units=n_steps_out))


GRU_GRU_reg.add(GRU(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act))
GRU_GRU_reg.add(GRU(units=neuron, activation=act))
GRU_GRU_reg.add(Dense(units=n_steps_out))


LSTM_LSTM_reg.add(LSTM(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act))
LSTM_LSTM_reg.add(LSTM(units=neuron, activation=act))
LSTM_LSTM_reg.add(Dense(units=n_steps_out))


LSTM_GRU_reg.add(LSTM(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act))
LSTM_GRU_reg.add(GRU(units=neuron, activation=act))
LSTM_GRU_reg.add(Dense(units=n_steps_out))


GRU_LSTM_reg.add(GRU(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act))
GRU_LSTM_reg.add(LSTM(units=neuron, activation=act))
GRU_LSTM_reg.add(Dense(units=n_steps_out))


# Compiling the RNNs

GRU_reg.compile(optimizer=opt,loss='mean_squared_error')
LSTM_reg.compile(optimizer=opt,loss='mean_squared_error')
GRU_GRU_reg.compile(optimizer=opt,loss='mean_squared_error')
LSTM_LSTM_reg.compile(optimizer=opt,loss='mean_squared_error')
LSTM_GRU_reg.compile(optimizer=opt,loss='mean_squared_error')
GRU_LSTM_reg.compile(optimizer=opt,loss='mean_squared_error')

DFS = Sequential()
CBGRU = Sequential()

DFS_GRU = Sequential()
CBLSTM = Sequential()

DFS_2LSTM = Sequential()
CB_2GRU = Sequential()


# filters defines how many features will be captured
# kernel size gives the size of the sliding window
DFS.add(Conv1D(filters=64, kernel_size=6, activation=act, input_shape=(X_train.shape[1],n_feature)))
DFS.add(MaxPooling1D(pool_size=4))
DFS.add(Dropout(0.2))  
DFS.add(LSTM(units=neuron, return_sequences=False, input_shape=(X_train.shape[1],n_feature), activation=act))
DFS.add(Dropout(0.190 + 0.0025 * n_steps_in))
DFS.add(Dense(units=n_steps_out))

CBGRU.add(Conv1D(filters=64, kernel_size=6, activation=act, input_shape=(X_train.shape[1],n_feature)))
CBGRU.add(MaxPooling1D(pool_size=4))
CBGRU.add(Dropout(0.2))  
CBGRU.add(Bidirectional(GRU(units=neuron, return_sequences=False, input_shape=(X_train.shape[1],n_feature), activation=act)))
CBGRU.add(Dense(units=n_steps_out))


DFS_GRU.add(Conv1D(filters=64, kernel_size=6, activation=act, input_shape=(X_train.shape[1],n_feature)))
DFS_GRU.add(MaxPooling1D(pool_size=4))
DFS_GRU.add(Dropout(0.2))  
DFS_GRU.add(GRU(units=neuron, return_sequences=False, input_shape=(X_train.shape[1],n_feature), activation=act))
DFS_GRU.add(Dropout(0.190 + 0.0025 * n_steps_in))
DFS_GRU.add(Dense(units=n_steps_out))

CBLSTM.add(Conv1D(filters=64, kernel_size=6, activation=act, input_shape=(X_train.shape[1],n_feature)))
CBLSTM.add(MaxPooling1D(pool_size=4))
CBLSTM.add(Dropout(0.2))  
CBLSTM.add(Bidirectional(LSTM(units=neuron, return_sequences=False, input_shape=(X_train.shape[1],n_feature), activation=act)))
CBLSTM.add(Dense(units=n_steps_out))


DFS_2LSTM.add(Conv1D(filters=64, kernel_size=6, activation=act, input_shape=(X_train.shape[1],n_feature)))
DFS_2LSTM.add(MaxPooling1D(pool_size=4))
DFS_2LSTM.add(Dropout(0.2))  
DFS_2LSTM.add(LSTM(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act))
DFS_2LSTM.add(LSTM(units=neuron,  activation=act))
DFS_2LSTM.add(Dropout(0.190 + 0.0025 * n_steps_in))
DFS_2LSTM.add(Dense(units=n_steps_out))

CB_2GRU.add(Conv1D(filters=64, kernel_size=6, activation=act, input_shape=(X_train.shape[1],n_feature)))
CB_2GRU.add(MaxPooling1D(pool_size=4))
CB_2GRU.add(Dropout(0.2))  
CB_2GRU.add(Bidirectional(GRU(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act)))
CB_2GRU.add(Bidirectional(GRU(units=neuron, activation=act)))
CB_2GRU.add(Dense(units=n_steps_out))



# Compiling the RNNs
adam = optimizers.Adam(lr=0.01)

DFS.compile(optimizer=adam,loss='mean_squared_error')
CBGRU.compile(optimizer=adam,loss='mean_squared_error')
DFS_2LSTM.compile(optimizer=adam,loss='mean_squared_error')
CBLSTM.compile(optimizer=adam,loss='mean_squared_error')
DFS_GRU.compile(optimizer=adam,loss='mean_squared_error')
CB_2GRU.compile(optimizer=adam,loss='mean_squared_error')

In [ ]:
RnnModelDict = {'LSTM': LSTM_reg, 'GRU': GRU_reg, 'LSTM_LSTM': LSTM_LSTM_reg, 'GRU_GRU': GRU_GRU_reg, 
                'LSTM_GRU': LSTM_GRU_reg, 'GRU_LSTM': GRU_LSTM_reg, 'DFS': DFS, 'CBGRU': CBGRU,
                'DFS_GRU': DFS_GRU, 'DFS_2LSTM': DFS_2LSTM, 'CB_2GRU': CB_2GRU, 'CBLSTM': CBLSTM}

X_test_24 = X_test[:24]
y_test_24 = y_test[:24]
rmse_df = pd.DataFrame(columns=['Model', 'train_rmse', 'valid_rmse', '24h_pred_rmse', 'train_time', ])


In [ ]:
for model in RnnModelDict:
    regressor = RnnModelDict[model]
    
    print('training start for', model)    
    start = time.process_time()
    regressor.fit(X_train,y_train,epochs=50,batch_size=32)
    train_time = round(time.process_time() - start, 2)
    
    print('results for training set')
    y_train_pred = regressor.predict(X_train)
    plot_predictions(y_train,y_train_pred)
    train_rmse = return_rmse(y_train,y_train_pred)
    
    print('results for valid set')
    y_valid_pred = regressor.predict(X_valid)
    plot_predictions(y_valid,y_valid_pred)
    valid_rmse = return_rmse(y_valid,y_valid_pred)
    
    
    print('results for test set - 24 hours')
    y_test_pred24 = regressor.predict(X_test_24)
    plot_predictions(y_test_24,y_test_pred24)
    test24_rmse = return_rmse(y_test_24,y_test_pred24)
    
    
    one_df = pd.DataFrame([[model, train_rmse, valid_rmse, test24_rmse, train_time]], 
                          columns=['Model', 'train_rmse', 'valid_rmse', '24h_pred_rmse', 'train_time'])
    rmse_df = pd.concat([rmse_df, one_df])

# save the rmse results 
rmse_df.to_csv('../rmse_result_all_v1.csv')


In [ ]:
history = regressor.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_valid, y_valid),
                        verbose=2, shuffle=False)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='valid')
plt.legend()
plt.show()

In [ ]:
# Transform back and plot
y_train_origin = y_train
y_train_origin = sc_y.inverse_transform(y_train_origin)

y_train_pred = regressor.predict(X_train)
y_train_pred_origin = sc_y.inverse_transform(y_train_pred)

plot_predictions(y_train_origin,y_train_pred_origin)
return_rmse(y_train_origin,y_train_pred_origin)